In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nltk
import pandas as pd
# from bertopic import BERTopic

# import gensim
# from gensim import corpora
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import re
from sentence_transformers import SentenceTransformer


/opt/anaconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
! pip install numpy

In [ ]:


! pip install numpy
! pip install pandas
! pip install nltk
! pip install matplotlib
! pip install sentence_transformers
! pip install berTopic





In [2]:
categories_to_add = ['Service Issue', "Food Quality", "Atmosphere", "Value for Money", "Hygiene","Food Options"]

In [4]:
df = pd.read_csv('new_dataset.csv')
df.drop(columns=categories_to_add,inplace=True)
df.head()

,Unnamed: 0,Review,Cleaned_Review,Cleaned_Name,Category,Severity,Is_Repeated,Frequency_Of_Complaint_Type,Urgency,Customer_Satisfaction,Review_Days,Response_Days,Others,Predicted_Category,Category_Label
0,0,"Stop eating at this place, I have visited bang...",stop eating place visited bangalores nd punes ...,pramod kumar,Atmosphere,High,False,Unique,Urgent,No Response,180.0,-1,0,['Food Quality' 'Atmosphere' 'Hygiene'],0
1,1,Food 2/5\nService 2/5\nAmbience 2/5 …,food service ambience,abhinav deep,Food Quality,Medium,False,Unique,Non-Urgent,High Satisfaction,365.0,365,0,['Atmosphere' 'Service Issue' 'Food Quality' '...,2
2,2,Idiotic varieties for the price they have char...,idiotic varieties price charged varieties boil...,vijay nammi,Value for Money,High,False,Unique,Urgent,No Response,30.0,-1,0,['Food Options'],5
3,3,"I am posting this live now, this is one of the...",posting live one worst places dont visit pathe...,surya ajay,Atmosphere,High,False,Unique,Urgent,High Satisfaction,365.0,365,0,['Service Issue' 'Food Quality' 'Atmosphere'],0
4,4,"We are pure vegetarians, I ordered veg biryani...",pure vegetarians ordered veg biryani swiggy go...,sai hithesh,Value for Money,Low,False,Unique,Non-Urgent,No Response,180.0,-1,0,['Food Quality'],5


In [ ]:
# from transformers import pipeline
# from datasets import Dataset
# import pandas as pd

# # Define Labels for Classification
# category_labels = ['Service Issue', 'Food Quality','Food Options',
#                    'Atmosphere', 'Value for Money', 'Hygiene']

# threshold = 0.5  # Adjust this value as needed

# # Load Zero-Shot Classification Model
# classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)

# # Convert DataFrame to Hugging Face Dataset
# dataset = Dataset.from_pandas(combined_df)

# # Function to Classify Each Review with Multi-Label Output
# def classify_review_batch(batch):
#     results = classifier(batch['Review'], candidate_labels=category_labels, multi_label=True)
#     batch['Predicted_Category'] = [
#         [label for label, score in zip(result['labels'], result['scores']) if score >= threshold]
#         if any(score >= threshold for score in result['scores'])
#         else [result['labels'][0]]  # Default to highest score if no category meets threshold
#         for result in results
#     ]
#     return batch

# # Apply Classification to the Dataset
# dataset = dataset.map(classify_review_batch, batched=True)

# # Convert back to DataFrame if needed
# combined_df = dataset.to_pandas()
# print(combined_df[['Review', 'Predicted_Category']].value_counts())
# combined_df.to_csv('new_classified_reviews.csv', index=False)

In [ ]:
# import pandas as pd
# from sklearn.preprocessing import MultiLabelBinarizer
# categories_to_add = ['Service Issue', "Food Quality", "Atmosphere", "Value for Money", "Hygiene", "Food Options"]
# mlb = MultiLabelBinarizer(classes=categories_to_add)
# one_hot_encoded = mlb.fit_transform(df['Predicted_Category'])

# # Create a DataFrame with the one-hot encoded values
# one_hot_df = pd.DataFrame(one_hot_encoded, columns=mlb.classes_)

# # Concatenate the original DataFrame with the one-hot encoded DataFrame
# final_df = pd.concat([df, one_hot_df], axis=1)

# print(final_df['Predicted_Category'])

In [ ]:
# # Initialize the MultiLabelBinarizer
# mlb = MultiLabelBinarizer(classes=categories_to_add)

# # Fit and transform the data
# one_hot_encoded = mlb.fit_transform(df['Predicted_Category'])

# # Create a new DataFrame with one-hot encoded data
# one_hot_df = pd.DataFrame(one_hot_encoded, columns=mlb.classes_)

# # Concatenate the original DataFrame with the one-hot encoded DataFrame
# final_df = pd.concat([df, one_hot_df], axis=1)

In [5]:
# Define categories to add
categories_to_add = ['Service Issue', 'Food Quality', 'Atmosphere', 'Value for Money', 'Hygiene', 'Food Options']




In [6]:
df['Predicted_Category'][0]

"['Food Quality' 'Atmosphere' 'Hygiene']"

In [7]:
# prompt: '['Food Quality' 'Atmosphere' 'Hygiene']' i want to convert this into individual columns of categories like ['Service Issue', 'Food Quality', 'Atmosphere', 'Value for Money', 'Hygiene', 'Food Options'] and put the binary values where the column is present in the givien string

def create_binary_columns(row):
  """Creates binary columns for each category based on the presence of the category in the string."""
  categories_found = row['Predicted_Category']
  for category in categories_to_add:
    if category in categories_found:
      row[category] = 1
    else:
      row[category] = 0
  return row

# Apply the function to create the new columns
df = df.apply(create_binary_columns, axis=1)

# Drop the original 'Predicted_Category' column if needed


df

,Unnamed: 0,Review,Cleaned_Review,Cleaned_Name,Category,Severity,Is_Repeated,Frequency_Of_Complaint_Type,Urgency,Customer_Satisfaction,...,Response_Days,Others,Predicted_Category,Category_Label,Service Issue,Food Quality,Atmosphere,Value for Money,Hygiene,Food Options
0,0,"Stop eating at this place, I have visited bang...",stop eating place visited bangalores nd punes ...,pramod kumar,Atmosphere,High,False,Unique,Urgent,No Response,...,-1,0,['Food Quality' 'Atmosphere' 'Hygiene'],0,0,1,1,0,1,0
1,1,Food 2/5\nService 2/5\nAmbience 2/5 …,food service ambience,abhinav deep,Food Quality,Medium,False,Unique,Non-Urgent,High Satisfaction,...,365,0,['Atmosphere' 'Service Issue' 'Food Quality' '...,2,1,1,1,0,0,1
2,2,Idiotic varieties for the price they have char...,idiotic varieties price charged varieties boil...,vijay nammi,Value for Money,High,False,Unique,Urgent,No Response,...,-1,0,['Food Options'],5,0,0,0,0,0,1
3,3,"I am posting this live now, this is one of the...",posting live one worst places dont visit pathe...,surya ajay,Atmosphere,High,False,Unique,Urgent,High Satisfaction,...,365,0,['Service Issue' 'Food Quality' 'Atmosphere'],0,1,1,1,0,0,0
4,4,"We are pure vegetarians, I ordered veg biryani...",pure vegetarians ordered veg biryani swiggy go...,sai hithesh,Value for Money,Low,False,Unique,Non-Urgent,No Response,...,-1,0,['Food Quality'],5,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4200,4200,Place where food lovers can enjoy their food a...,place food lovers enjoy food apart dishes rest...,hemanth kumar pallapu,Food Quality,Low,False,Unique,Non-Urgent,No Response,...,-1,0,['Value for Money' 'Food Quality' 'Food Option...,2,0,1,1,1,0,1
4201,4201,"Good place to visit with friends and family, t...",good place visit friends family customised foo...,abhinay maredugonda,Food Quality,Low,False,Unique,Non-Urgent,No Response,...,-1,0,['Value for Money' 'Atmosphere' 'Food Options'...,2,0,1,1,1,0,1
4202,4202,"The food is very good, service is also good. B...",food good service also good compare service co...,devesh gupta,Food Quality,Low,False,Unique,Non-Urgent,No Response,...,-1,0,['Food Quality' 'Value for Money' 'Food Options'],2,0,1,0,1,0,1
4203,4203,Nice place to hangout with a small team of upt...,nice place hangout small team upto members goo...,raja roy,Food Quality,Low,False,Unique,Non-Urgent,No Response,...,-1,0,['Value for Money' 'Atmosphere' 'Food Quality'...,2,1,1,1,1,0,1


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
import numpy as np



# Split the data into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['Cleaned_Review'], df[categories_to_add], test_size=0.2, random_state=42
)


# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the texts
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels))

! pip install transformers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
import numpy as np

# ... (rest of the code remains the same)

# Load BERT model
# Change num_labels to match the number of categories
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(categories_to_add))

# Compile the model
# Change loss to BinaryCrossentropy for multi-label classification
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), # Changed loss function
              metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset.shuffle(100).batch(16), epochs=4, batch_size=16,
                    validation_data=test_dataset.batch(16))


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/4
211/211 [==============================] - 1010s 5s/step - loss: 0.5499 - accuracy: 0.4893 - val_loss: 0.4715 - val_accuracy: 0.6005
Epoch 2/4
211/211 [==============================] - 1084s 5s/step - loss: 0.4551 - accuracy: 0.6052 - val_loss: 0.4437 - val_accuracy: 0.5910
Epoch 3/4
211/211 [==============================] - 1014s 5s/step - loss: 0.3862 - accuracy: 0.6189 - val_loss: 0.4465 - val_accuracy: 0.5589
Epoch 4/4
211/211 [==============================] - 945s 4s/step - loss: 0.3190 - accuracy: 0.6356 - val_loss: 0.4576 - val_accuracy: 0.5482


In [ ]:
# prompt: want to increase the accuracy of code present in above cell

import numpy as np
import matplotlib.pyplot as plt
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score, f1_score

# ... (rest of the code remains the same)

# Load BERT model
# Change num_labels to match the number of categories
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(categories_to_add))

# Compile the model
# Change loss to BinaryCrossentropy for multi-label classification
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), # Changed loss function
              metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset.shuffle(100).batch(16), epochs=5, batch_size=16,
                    validation_data=test_dataset.batch(16))


# Make predictions on the test set
predictions = model.predict(test_dataset.batch(16))
predicted_labels = (tf.nn.sigmoid(predictions.logits) > 0.5).numpy().astype(int)

# Calculate evaluation metrics
print(classification_report(test_labels, predicted_labels, target_names=categories_to_add))
print("Accuracy:", accuracy_score(test_labels, predicted_labels))
print("F1 Score (Micro):", f1_score(test_labels, predicted_labels, average='micro'))
print("F1 Score (Macro):", f1_score(test_labels, predicted_labels, average='macro'))

In [ ]:
# prompt: as well as i want predict multi label for a specific unseen review

def predict_multi_label(review_text):
  """Predicts the multi-label categories for a given review text."""

  # Tokenize the review text
  inputs = tokenizer(review_text, truncation=True, padding=True, max_length=128, return_tensors='tf')

  # Make predictions using the trained model
  outputs = model(inputs)
  predictions = tf.nn.sigmoid(outputs.logits).numpy()

  # Define a threshold for classifying labels
  threshold = 0.5  # Adjust as needed
  
  # Get the predicted labels based on the threshold
  predicted_labels = [label for i, label in enumerate(categories_to_add) if predictions[0][i] >= threshold]
  if predict_multi_label:
    return predicted_labels
  else:
    return max(predicted_labels, key=lambda x: predictions[0][categories_to_add.index(x)])

# Example usage:
unseen_review = "There is no cleanliness and staff was not good at all"
predicted_categories = predict_multi_label(unseen_review)
print(f"Predicted categories for the review: {predicted_categories}")

In [ ]:
# prompt: how to use this folder for further predictions in another location

# ... (Your existing code)

# Save the trained model and tokenizer
model.save_pretrained('/content/drive/MyDrive/your_model_folder') # Change to your desired folder
tokenizer.save_pretrained('/content/drive/MyDrive/your_model_folder')

# Function to load and use the model for predictions in another location
def load_and_predict(review_text, model_path='/content/drive/MyDrive/your_model_folder'):
  """Loads a saved model and tokenizer from Google Drive and makes predictions."""

  loaded_model = TFBertForSequenceClassification.from_pretrained(model_path)
  loaded_tokenizer = BertTokenizer.from_pretrained(model_path)

  # Tokenize the review text
  inputs = loaded_tokenizer(review_text, truncation=True, padding=True, max_length=128, return_tensors='tf')

  # Make predictions using the loaded model
  outputs = loaded_model(inputs)
  predictions = tf.nn.sigmoid(outputs.logits).numpy()

  # Define a threshold for classifying labels
  threshold = 0.7  # Adjust as needed

  # Get the predicted labels based on the threshold
  predicted_labels = [label for i, label in enumerate(categories_to_add) if predictions[0][i] >= threshold]

  return predicted_labels


# Example usage in another location:
# (Assume you've mounted Google Drive in the new location)

# new_review = "The food was delicious, but the service was slow."
# predicted_categories = load_and_predict(new_review)
# print(f"Predicted categories for the new review: {predicted_categories}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')